In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import InputFiles as I, Calculators as C, Datasets as D, Parsers as P, Utilities as U
from mppi.Utilities import Tools
from mppi.Utilities import Constants as Const
import matplotlib.pyplot as plt
import numpy as np
import os, yaml

In [ ]:
dry_run = False
rsync_str = "rsync -rLptgoDzv --exclude={'*_fragment_*','*_fragments_*','*wfc*','*.dat'} "
if dry_run: rsync_str += '--dry-run '
ssh_str = '-e ssh ismhpc:/work/dalessandro/2D_ELECTRONIC_OPTICAL_PROPERTIES/GERMANENE/GW_HF_analysis/ '
dest_path = 'GW_HF_analysis'
dest_path = os.path.abspath(dest_path)
dest_path = "'%s'"%dest_path
command = rsync_str+ssh_str+dest_path
print(command)

In [3]:
omp = 1
mpi = 16 #4
sbatch_options = ['--partition=all12h']

In [4]:
#code = C.YamboCalculator(omp=omp,mpi=mpi,executable='yambo')
code = C.YamboCalculator(omp=omp,mpi=mpi,executable='yambo',scheduler='slurm',sbatch_options=sbatch_options)
code.global_options()

Initialize a Yambo calculator with scheduler slurm


{'omp': 1,
 'mpi': 16,
 'mpi_run': 'mpirun -np',
 'executable': 'yambo',
 'scheduler': 'slurm',
 'skip': True,
 'clean_restart': True,
 'dry_run': False,
 'wait_end_run': True,
 'sbatch_options': ['--partition=all12h'],
 'activate_BeeOND': True,
 'verbose': True}

# GW convergence analysis of the Germanene

We compute the GW corrections of the DFT band structure of the Germanene. 

We discuss the convergene procedure using various approximations of the self-energy.

We have performed various preliminary tests in particular:

__Parallelization strategy__

We set the SE_ROLEs as 'b.qp.q' and we observe that (with 16 mpi)
* for 6,6,1 kpoints the highest performance are achieved with '16.1.1'
* for 12,12,1 kpoints the only working configuration is '16.1.1' (reasons are to be checked further...)

__Random integration method (RIM) and Coulomb potential cutoff__

.........

## Hartree-Fock convergence analysis 

At the Hartree-Fock (HF) level the self energy is limited to the exchange part $\Sigma_x$ written in term of the
bare Coulomb potential.

The diagonal matrix elements of $\Sigma_x$ on the KS states are written as
$$
\langle nk|\Sigma_x|nk \rangle = -\sum_{n'}\int_{BZ}dq\sum_G \frac{4\pi}{|q+G|^2}
|\rho_{nn'}(k,q,G|^2f_{n',k-q}
$$
The sum over $n'$ il limited to the full bands due the $f_{n'}$ factor and so the self energy in this
approximation scheme depends on the $k$-points sampling in the BZ (q sum) and on the cutoff of the plane wave
expansion of the KS states.

We study the converge of the direct gap at the $K$ point in function of the number of plane wave of the exchange (EXXRLvcs variable in Yambo).
The highest value of this variable is given by the cutoff of the density matrix in QuantumESPRESSO, that in turn is
4 times the cutoff on the KS wave functions ($90*4=360$ Ry in the present analysis).

We also study the convergence of the gap in function of the $k$-points sampling of the BZ.

In [5]:
source_dir = 'DFT_analysis/kpoints_12-nbnds_60.save'
run_dir = 'GW_HF_analysis/kpoints_12'

In [6]:
Tools.build_SAVE(source_dir,run_dir,command='p2y')

SAVE folder already present in GW_HF_analysis/kpoints_12. No operations performed.


We build the input for HF analysis including the random integration method (RIM) for the small $q$ component of the Coulomb potential
and the cutoff of the Coulomb potential in the off plane direction. The z dimension of the simulation box is 
$7.643*8 = 61.44$ au and we set the cutoff of the potential slightly smaller than this value.

In [12]:
cut_z = 60
RandQpts = 1000000 # Number of random q-points in the BZ
RandGvec = 100 # use the RIM for the first 100 G-components of the coulomb potential 

dft_data = P.PwParser(os.path.join(source_dir,'data-file-schema.xml'))
K_value = len(dft_data.kpoints) # K is the last point in the k sampling
start_band,stop_band = 28,29 # bands range for the computation of the HF corrections

#EXXRLvcs = [20,40,60,80,100,200]
EXXRLvcs = [200]

Parse file : DFT_analysis/kpoints_12-nbnds_60.save/data-file-schema.xml


In [13]:
inp = I.YamboInput(args='yambo -r -x -V all',folder=run_dir)
inp.set_extendOut()
inp.set_kRange(K_value,K_value)
inp.set_bandRange(start_band,stop_band)
inp['variables']['RandQpts'] = [RandQpts,'']
inp['variables']['RandGvec'] = [RandGvec,'']
inp['variables']['CUTGeo'] = 'box z'
inp['variables']['CUTBox'] = [[0.,0.,cut_z],'au']

inp['variables']['SE_CPU'] = '%s.1.1'%mpi
inp['variables']['SE_ROLEs'] = 'b.qp.q'
#inp

In [14]:
study = D.Dataset(run_dir=run_dir,num_tasks=3)

Initialize a Dataset with 3 parallel tasks


In [15]:
for ex in EXXRLvcs:
    name = 'EXXRLvcs_%s_rim-cut_%s'%(ex,cut_z)
    idd = {'EXXRLvcs':ex,'cutz':cut_z}
    inp['variables']['EXXRLvcs'] = [ex,'Ry']
    study.append_run(id=idd,input=inp,runner=code)

In [16]:
study.run()

Run the selection [0] with the parallel task_groups [[0]] 

Run the task [0] 
run command: mpirun -np 16 yambo -F EXXRLvcs_200-cutz_60.in -J EXXRLvcs_200-cutz_60 -C EXXRLvcs_200-cutz_60
slurm submit:  cd GW_HF_analysis/kpoints_12 ; sbatch job_EXXRLvcs_200-cutz_60.sh
computation EXXRLvcs_200-cutz_60 is running...
computation EXXRLvcs_200-cutz_60 ended
Run performed in 16m-04s
Task [0] ended 
 


{0: {'output': ['GW_HF_analysis/kpoints_12/EXXRLvcs_200-cutz_60/o-EXXRLvcs_200-cutz_60.hf'],
  'report': 'GW_HF_analysis/kpoints_12/EXXRLvcs_200-cutz_60/r-EXXRLvcs_200-cutz_60_HF_and_locXC_rim_cut',
  'dft': 'GW_HF_analysis/kpoints_12/SAVE/ns.db1',
  'HF_and_locXC': 'GW_HF_analysis/kpoints_12/EXXRLvcs_200-cutz_60/ndb.HF_and_locXC',
  'RIM': 'GW_HF_analysis/kpoints_12/EXXRLvcs_200-cutz_60/ndb.RIM',
  'cutoff': 'GW_HF_analysis/kpoints_12/EXXRLvcs_200-cutz_60/ndb.cutoff'}}

In [ ]:
###########################################################################

In [13]:
results = code.run(run_dir=run_dir,input=inp,name=name,skip=True)
results

run command: mpirun -np 16 yambo -F hf_5_rim_cut40.in -J hf_5_rim_cut40 -C hf_5_rim_cut40
slurm submit:  cd GW_HF_analysis/kpoints_12 ; sbatch job_hf_5_rim_cut40.sh
computation hf_5_rim_cut40 is running...
computation hf_5_rim_cut40 ended
Run performed in 25m-38s


{'output': ['GW_HF_analysis/kpoints_12/hf_5_rim_cut40/o-hf_5_rim_cut40.hf'],
 'report': 'GW_HF_analysis/kpoints_12/hf_5_rim_cut40/r-hf_5_rim_cut40_HF_and_locXC_rim_cut',
 'dft': 'GW_HF_analysis/kpoints_12/SAVE/ns.db1',
 'HF_and_locXC': 'GW_HF_analysis/kpoints_12/hf_5_rim_cut40/ndb.HF_and_locXC',
 'RIM': 'GW_HF_analysis/kpoints_12/hf_5_rim_cut40/ndb.RIM',
 'cutoff': 'GW_HF_analysis/kpoints_12/hf_5_rim_cut40/ndb.cutoff'}

In [ ]:
###########################################################

In [52]:
dft = P.YamboDftParser(os.path.join(run_dir,'SAVE','ns.db1'))

Parse file : GW_HF_analysis/kpoints_12/SAVE/ns.db1


In [53]:
dft.kpoints

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.08333333,  0.        ],
       [ 0.        ,  0.16666667,  0.        ],
       [ 0.        ,  0.25      ,  0.        ],
       [ 0.        ,  0.33333333,  0.        ],
       [ 0.        ,  0.41666667,  0.        ],
       [ 0.        , -0.5       ,  0.        ],
       [ 0.08333333,  0.125     ,  0.        ],
       [ 0.08333333,  0.20833333,  0.        ],
       [ 0.08333333,  0.29166667,  0.        ],
       [ 0.08333333,  0.375     ,  0.        ],
       [ 0.08333333,  0.45833333,  0.        ],
       [ 0.16666667,  0.25      ,  0.        ],
       [ 0.16666667,  0.33333333,  0.        ],
       [ 0.16666667,  0.41666667,  0.        ],
       [ 0.16666667,  0.5       ,  0.        ],
       [ 0.25      ,  0.375     ,  0.        ],
       [ 0.25      ,  0.45833333,  0.        ],
       [ 0.33333333,  0.5       ,  0.        ]])

In [51]:
dft.lattice/7.643

array([[ 1.       ,  0.       ,  0.       ],
       [-0.5      ,  0.8660254,  0.       ],
       [ 0.       ,  0.       ,  8.       ]])

In [48]:
dft.alat

7.643

In [11]:
inp = I.YamboInput(args='yambo -x -V rl',folder=run_dir)
inp

{'args': 'yambo -x -V rl',
 'folder': 'GW_analysis',
 'filename': 'yambo.in',
 'arguments': ['HF_and_locXC'],
 'variables': {'FFTGvecs': [52407.0, 'RL'],
  'SE_Threads': [0.0, ''],
  'EXXRLvcs': [356277.0, 'RL'],
  'VXCRLvcs': [356277.0, 'RL'],
  'QPkrange': [[1, 19, 1, 60], '']}}